In [2]:
import azure.cosmos.cosmos_client as cosmos_client

from azure.cosmos import CosmosClient, exceptions, PartitionKey
endpoint = ""
key = ""

client = CosmosClient(endpoint, credential={'masterKey': key})

In [3]:
try:
    # List the databases in the Cosmos account
    database_list = list(client.list_databases())
    if database_list:
        print("Connected successfully.")
        print("List of databases:")
        for database in database_list:
            print(database['id'])
    else:
        print("Connected successfully, but no databases were found.")
except exceptions.CosmosHttpResponseError as e:
    print(f"Failed to connect: {e.status_code} {e.message}")


Connected successfully.
List of databases:
temps
ToDoList


In [4]:
import json
from datetime import datetime

In [20]:
client = cosmos_client.CosmosClient(endpoint, {'masterKey': key})
database = client.get_database_client('temps')
container = database.get_container_client('temps')
dashboard_container = database.get_container_client('dashboard')

In [21]:
# temps_container = database.get_container_client('temps')

# query = "SELECT * FROM c"

# readings = list(temps_container.query_items(
#     query=query,
#     enable_cross_partition_query=True
# ))

# request_charge = dashboard_container.client_connection.last_response_headers['x-ms-request-charge']

# print('Query returned {0} items. Operation consumed {1} request units'.format(len(readings), request_charge))

Query returned 3135 items. Operation consumed 5.43 request units


In [30]:
import time
import base64

temps_container = database.get_container_client('temps')

# 初始化上一次查詢的最後一個項目
last_item = None

while True:
    query = "SELECT * FROM c"
    
    # 執行 SQL 查詢
    readings = list(temps_container.query_items(query=query, enable_cross_partition_query=True))
    
    # 如果有新的項目，則刷新
    if readings and readings[-1] != last_item:
        last_item = readings[-1]
        encoded_data = dict(last_item)['Body']
        decoded_data = base64.b64decode(encoded_data).decode('utf-8')

        print('Latest Reading:', decoded_data)

    # 獲取請求單位
    request_charge = temps_container.client_connection.last_response_headers['x-ms-request-charge']
    
    # 等待一段時間再執行下一次查詢（每隔3秒）
    time.sleep(3)

# 輸出查詢結果和請求單位
print('Query returned {0} items. Operation consumed {1} request units'.format(len(readings), request_charge))

Latest Reading: {
  "version" : "4.0",
  "id" : 831,
  "@timestamp" : "2024-02-29T03:53:41.636Z",
  "sensorId" : "claireye-0",
  "objects" : [
    "-1|56.6174|297.208|131.616|510.333|Person"
  ]
}
Latest Reading: {
  "version" : "4.0",
  "id" : 2463,
  "@timestamp" : "2024-02-29T03:54:36.036Z",
  "sensorId" : "claireye-0",
  "objects" : [
    "-1|1204.86|376.872|1235.86|451.202|Person",
    "-1|742.494|411.68|782.877|512.926|Person"
  ]
}
Latest Reading: {
  "version" : "4.0",
  "id" : 3555,
  "@timestamp" : "2024-02-29T03:55:12.436Z",
  "sensorId" : "claireye-0",
  "objects" : [
    "-1|1110.65|369.081|1147.69|457.388|Person"
  ]
}


KeyboardInterrupt: 

In [22]:
readings[2000]

{'id': '26498d83-6533-4dd6-8326-20ede319ba2d',
 'temps': 'myEdgeDevice-2024-02',
 'Properties': {},
 'SystemProperties': {'iothub-connection-device-id': 'myEdgeDevice',
  'iothub-connection-auth-method': '{"scope":"device","type":"sas","issuer":"iothub"}',
  'iothub-connection-auth-generation-id': '638439253661197147',
  'iothub-enqueuedtime': '2024-02-29T02:04:35.613Z',
  'iothub-message-source': 'Telemetry'},
 'iothub-name': 'icam500',
 'Body': 'ewogICJ2ZXJzaW9uIiA6ICI0LjAiLAogICJpZCIgOiA1MjgsCiAgIkB0aW1lc3RhbXAiIDogIjIwMjQtMDItMjlUMDI6MDQ6NDguMjQ4WiIsCiAgInNlbnNvcklkIiA6ICJjbGFpcmV5ZS0wIiwKICAib2JqZWN0cyIgOiBbCiAgICAiLTF8Mjc5Ljg0fDMxOC4wNTF8Mjk2Ljg2MXwzNjQuNDExfFBlcnNvbiIsCiAgICAiLTF8NDQuNDIzOHwyOTUuNzM3fDEyOS45MDd8NTM0LjE0fFBlcnNvbiIKICBdCn0=',
 '_rid': 'oXU7ANpIeFbRBwAAAAAAAA==',
 '_self': 'dbs/oXU7AA==/colls/oXU7ANpIeFY=/docs/oXU7ANpIeFbRBwAAAAAAAA==/',
 '_etag': '"1d025973-0000-0200-0000-65dfe6330000"',
 '_attachments': 'attachments/',
 '_ts': 1709172275}

In [29]:
import base64

body_message = dict(readings[2000])['Body']
encoded_data = body_message

decoded_data = base64.b64decode(encoded_data).decode('utf-8')

print(decoded_data)


{
  "version" : "4.0",
  "id" : 528,
  "@timestamp" : "2024-02-29T02:04:48.248Z",
  "sensorId" : "claireye-0",
  "objects" : [
    "-1|279.84|318.051|296.861|364.411|Person",
    "-1|44.4238|295.737|129.907|534.14|Person"
  ]
}
